In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
directories = {
    "SSC": "../ssc",
    "DVS Gesture": "../dvs_gesture",
    "HSD": "../hsd",
}

collected_frames = dict()
for dataset, path in directories.items():
    df = pd.read_csv(Path(path) / "times_new_table.csv", header=0, index_col=0)
    df["dataset"] = dataset
    collected_frames[dataset] = df

times = pd.concat(collected_frames.values(), ignore_index=True)

In [ ]:
# sns.set(font_scale=1.5, style="white")
time_no_reset = times[times.step != "reset"]
for dataset in time_no_reset.dataset.unique():
    plt.figure()
    plot = sns.barplot(data=time_no_reset[time_no_reset.dataset == dataset], y="time", x="step", hue="algorithm")
    plot.set_xlabel("Time per epoch (s)")
    plot.axes.yaxis.label.set_visible(False)
    plt.tight_layout()
    plt.savefig(f"comp_time_comparison_{dataset}.svg")

In [ ]:
for dataset, df_ds in collected_frames.items():
    print(dataset)
    for step in ["forward", "backward"]:
    # for step in ["backward"]:
        print(step)
        df = df_ds[df_ds.step == step]
        grouped = df.groupby(["algorithm"])
        means = grouped.mean() # .reset_index()
        stds = grouped.std()
        fig = plt.figure(figsize=(2.5, 3))
        x = ['BPTT', 'SLAYER', 'EXODUS']
        y = [means.loc[method]["time"]*1000 for method in x]
        s = [stds.loc[method]["time"]*1000 for method in x]
        b = np.array(y[1:])
        db = np.array(s[1:])
        ratios = y[0] / b
        s_ratios = np.sqrt(s[0]**2 / b**2 + y[0]**2 * db**2 / b**4)
        print(f"\tBPTT: {y[0]:.2f} +- {s[0]:.1f} ms")
        print(f"\tSLAYER: {y[1]:.2f} +- {s[1]:.1f} ms")
        print(f"\t\t {ratios[0]:.2f} +- {s_ratios[0]:.2f} times faster")
        print(f"\tEXODUS: {y[2]:.2f} +- {s[2]:.1f} ms")
        print(f"\t\t {ratios[1]:.2f} +- {s_ratios[1]:.2f} times faster")
        # barlist=plt.bar(x, y, yerr=s) #, width=1)
        ratios = np.insert(ratios, 0, 1.)
        s_ratios = np.insert(s_ratios, 0, 0)
        barlist=plt.bar(x, ratios, yerr=s_ratios) #, width=1)
        barlist[0].set_color('C2')
        barlist[1].set_color('C1')
        barlist[2].set_color('C0')
        ax = fig.get_axes()[0]
        ax.set_xticklabels(['BPTT', 'SLAYER', 'EXODUS'], rotation=20)
        ax.set_ylabel("Backward pass speedup")
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        plt.tight_layout()
        plt.savefig(f"time_{dataset}_{step}.svg")
    
    

In [ ]:
np.array([11.22, 14.09, 14.59]).mean()

In [ ]:
np.array([17.30, 16.57, 9.1]).mean()